### Algebraic Evaluation of Coupled-Cluster Diagrams - Application

The code to evaluate the coupled-cluster diagrams starts with the exponential expansion of a sum of amplitude operators, for example, $e^{T_1 + T_2}$. To accomplish this we use Sympy. The program we are using is called __cc-ade__ and we start with, to get eg the doubles amplitude at single & doubles level

In [1]:
import sys
sys.path.append('../cc-ade')
from cc_ade import expansion

SD = expansion('SD', 'D')

The first thing we can get back is a Latex representation of the expansion from __.do_latex()__ 

In [2]:
latex_SD = SD.do_latex()
print('SD doubles = ', latex_SD)

SD doubles =  $\frac{1}{24}T_1^{4} + \frac{1}{6}T_1^{3} + \frac{1}{2}T_1^{2}T_2 + \frac{1}{2}T_1^{2} + T_1T_2 + T_1 + \frac{1}{2}T_2^{2} + T_2 + 1 $


SD doubles =  $\frac{1}{24}T_1^{4} + \frac{1}{6}T_1^{3} + \frac{1}{2}T_1^{2}T_2 + \frac{1}{2}T_1^{2} + T_1T_2 + T_1 + \frac{1}{2}T_2^{2} + T_2 + 1 $


We can get information about the terms from __.do_term(id)__. Each term is identified by a string of four characters where the entry in the ith position indicates the power of the ith amplitude in the term (0 indicates amplitude does not appear in term), so $T_1^2 T_3$ would be identified with '2010'. We are only coding for upto $T_4$ hence the four characters.

In [3]:
SD.do_term('2100')

Level is [ SD ]  Order is [ 4 ]
Term id  [ 2100 ]   Multiplier is [  1 / 2  ]
Latex string is [  \frac{1}{2} T_1^{2} T_2   ]
Vertex string is [  +- | +- | ++--  ]
Interaction number is [  - 2  ]

Contractor               Contractions
hhpp               ['+|+|--', '+|+-|-', '+-|-|+', '+|-|+-', '-|-|++']


In our approach each amplitude is represented by a sign-string (s-string), for example, $T_3$ is __+++---__. A term ,which is product of amplitudes, is written as __+-|++--__ with a __|__ symbol seperating the amplitudes. We see above above for '2100' the term s-string is __+-|+-|++--__. The _interaction number_ of the term is the target excitation level (for doubles this is 2) minus the actual excitation level of the term (number of __+__ signs, for '2100' this is 4). _Contractor_ are the Hamiltonian elements that can contract with the term, that is those with an _interaction number_ equal to the _interaction number_ of the term. The _contractions_ are all the valid ways in which the Hamiltonian can contract with the term. It is possible to get the Hamiltonian definitions from __.vertex__ and __.interaction__

In [4]:
print('2-body Hamiltonian hhpp ', SD.vertex['hhpp'], SD.interaction['hhpp'])

2-body Hamiltonian hhpp  ['+', '+', '-', '-'] [-2]


There is a __.get(type, source, id='')__ method which can be used as __.get('t', term_id)__ to get the internal representation of a term. This returns a list as \[term_id, factor, amplitude powers, s-strings, interaction number \]. It can also be used as __.get('i', interaction_number)__ which returns all Hamiltonian elements with the requested _interaction number_ as a list. If the list obtained by get('i') is passed to __.get('v', hamiltonian_list)__ then a list of the Hamiltonian s-strings is returned.

In [5]:
print(SD.get('t', '2100'))
print(SD.get('i', -1))
print(SD.get('v', SD.get('i', -1)))

['2100', [1, 2], [2, 1, 0, 0], [['+', '-'], ['+', '-'], ['+', '+', '-', '-']], -2]
['hp', 'phpp', 'hhhp']
[['+', '-'], ['+', '+', '-'], ['+', '-', '-']]


For each term in the expansion, each possible Hamiltonian contraction corresponds to a diagram. The overall sign and multiplying factor of the diagram can determined from the s-string of the Hamiltonian operator. To determine the the permutation of external lines the diagram must be labelled. The labelling algorithms are coded in the _lablib_ module. The labelling of diagrams is not unique so _lablib_ is just an example of possible schemes. The _lablib_ module is not optimized in any way, for example Hamiltonian elements +-- and -++ are just mirrors of each other and could be parameterised in a single routine. The routines not computationally intensive so this has not been done incidentally allowing for clearer code. Each diagram is stored as a dictionary in a list in the class variable __.ade__. Although this demonstration program is coded for S, D, T and Q the _lablib_ routines are designed to be applicable for any order of cluster expansion. They have been extensively tested upto Q level (see the section on validation)

__.get('r', reference)__ will return the __.ade__ list item matching the Shavitt & Bartlett reference supplied. __.get('c', s-string)__ will return the __.ade__ list item matching the s-string supplied. The s-string is of the form Hamiltonian contraction s-string __:__ term amplitudes s-string, so for the '2100' example in \[3\] one Hamitonian contraction s-string could be '+|+|--' and the term s-string is '+-|+-|++--' so we would pass '+|+|--:+-|+-|++--'.

In [6]:
print(SD.get('r', 'D_{9}')['code'])
print()
print(SD.get('c', '+|+|--:+-|+-|++--'))

0.25 * np.einsum('klcd,ci,dj,ak,bl->abij', gs[o,o,v,v], ts, ts, ts, ts)

{'id': '2100(hhpp)', 'labels': ('klcd', 'ci|dj|abkl'), 'sign': (1, [4, 2]), 'factor': 2, 'sigma': 'dklc', 'permutation': '\\hat{P}(ij) ', 'latex': '\\frac{1}{4}\\hat{P}(ij)  \\displaystyle \\sum_{dklc} \\langle kl\\Vertcd \\rangle t^{c}_{i} t^{d}_{j} t^{ab}_{kl}', 'code': "0.25 * np.einsum('klcd,ci,dj,abkl->abij', gs[o,o,v,v], ts, ts, td)", 'reference': 'D_{7a}', 'contraction': '+|+|--'}


The diagrams are computed in a list of dictionaries in __.ade__. For each diagram there is a list entry with keys 
+ _id_ - '4000(hhpp)' - the term id and the Hamiltonian contraction
+ _labels_ -  ('klcd', 'ci|dj|ak|bl') - tuple (operator label, amplitude labels)
+ _sign_ - (1, \[4, 2\]) - tuple (sign, \[hole lines, loops\])
+ _factor_ - 2 - number of $\frac{1}{2}$ multipliers
+ _sigma_ - lkdc - summation over these indices
+ _permutation_ - $\hat{P}(ab) \hat{P}(ij)$ - latex representation of permutations
+ _latex_ - $\frac{1}{4}\hat{P}(ab) \hat{P}(ij)  \displaystyle \sum_{lkdc} \langle kl\Vert cd \rangle t^{c}_{i} t^{d}_{j} t^{a}_{k} t^{b}_{l}$ - latex representation of diagram algebra
+ _code_ - "0.25 * np.einsum('klcd,ci,dj,ak,bl->ijab', gs[o,o,v,v], ts, ts, ts, ts)" - Python einsum expression.
+ _reference_ - $D_{9}$ - Shavitt & Bartlett reference for diagram
+ _contraction_ - '+|+|-|-' - s-string of Hamiltonian

There is no obvious order for the diagram so they are listed in arbitrary order. There is a __.order_by_reference()__ method which will order the __.ade()__ list by Shavitt and Bartlett reference.

In [7]:
SD.order_by_reference()
SD.ade[0]

{'id': '0000(pphh)',
 'labels': ('abij', ''),
 'sign': (1, [2, 2]),
 'factor': 0,
 'sigma': '',
 'permutation': '',
 'latex': ' \\langle ab\\Vertij \\rangle',
 'code': "np.einsum('abij->abij', gs[v,v,o,o])",
 'reference': 'D_{1}',
 'contraction': '0'}

### Validation
In order to verify the diagrams computed in _ade_ there is a _validation_ suite. This consists of a _reference.txt_ file which contains the Python einsum expressions for the diagrams as determined from Shavitt & Bartlett. The validation process is initiated with a call to __validate(ade_object, test_set)__. The _test\_set_ is either 
+ 'c' - singles, doubles, triples and quadruples for single carbon atom in 3-21g basis. __unconverged amplitudes for testing only!__
+ 'hf'- hydrogen fluoride in singles, doubles and triples in 6-31g basis (r=1.6 bohr) 

The _validate_ module has a __.compare(type)__ method. _Type_ can be _'all'_ in which case every diagram in the _.ade_ list will be compared with the Shavitt & Bartlett definition. If _type_ is a Shavitt & Bartlett reference then that diagram will be compared. If _type_ is a Hamiltonian id then all contractions of that Hamiltonian will be compared. The output is, for example

In [8]:
from validation.validate import validate
test = validate(SD, 'c')
test.compare('hhpp')

D_{3a}    ++|--        0.018753199846591   0.018753199846591  True True
D_{3b}    +-|+-        0.012007054940035   0.012007054940035  True True
D_{3c}    ++-|-        0.030511793264890   0.030511793264890  True True
D_{3d}    +|+--        0.015795670617637   0.015795670617637  True True
D_{7a}    +|+|--       0.000000811094776   0.000000811094776  True True
D_{7b}    -|-|++       0.000001579422770   0.000001579422770  True True
D_{7c}    +|-|+-       0.000000609290951   0.000000609290951  True True
D_{7d}    +|+-|-       0.000001370808451   0.000001370808451  True True
D_{7e}    +-|-|+       0.000000339310896   0.000000339310896  True True
D_{9}     +|+|-|-      0.000000000065099   0.000000000065099  True True


The output shows the Shavitt and Bartlett reference, the Hamiltonian contraction, np.linalg.norm of Shavitt & Bartlett and _ade_ computations, booleans are comparisons of norms followed by np.allclose on Shavitt & Bartlett and _ade_ computed amplitudes. For triples and quadruples these comparisons can take some time to run - quadruples are an 8 deep loop! The results of the carbon and hydrogen fluoride validations are given in the files 'c-3-21g-sdtq.txt' and 'hf-6-31g-sdt.txt'.

The _validate_ module contains code to do the permutations where you can pass eg a string like 'ijklabcd->a/b/cd' to a subroutine to obtain the requested permutation. Although the idea of this exercise is just to generate the algebraic form of the diagrams the _validate_ module provides an __.evaluate(reference, data)__ method which will evaluate a Shavitt & Bartlett type diagram reference given spin-orbital data. As an example we could calculate the CCD energy correction for HF using the saved .npz file in the _validation_ directory

In [9]:
import numpy as np
from validation.validate import validate

#load saved data 
data = np.load('../cc-ade/validation/hf.npz')
nvir, nocc = data['ts'].shape                               #get the occupied and virtual orbital numbers
o, v, n = slice(None, nocc), slice(nocc, None), np.newaxis  #make slices for occupied and virtual orbitals

#user_data are variable and values dictionary passed to eval
user_data = {}
user_data['gs'], user_data['fs'] = data['g'], data['f']     #spin eri and fock
user_data['td'] = np.zeros_like(data['td'])                 #only need doubles amplitudes, initialise to zero
user_data['o'], user_data['v'] = o, v                       #orbital slices

#fock denominator
eps = np.diag(user_data['fs'])
d   = -eps[v,n,n,n] - eps[n,v,n,n] + eps[n,n,o,n] + eps[n,n,n,o]

#run parameters
max_cycle, tol, norm, converged = 50, 1e-6, 0, False

#get diagrams for doubles at doubles level
D = expansion('D', 'D')
V = validate(D, 'hf')                                        #need instance of validate for evaluate routine

#iteration
for cycle in range(0, max_cycle):
    
    #compute double amplitudes for this cycle
    t = np.zeros_like(user_data['td'])
    for diagram in D.ade:
        t += V.evaluate(diagram['reference'], user_data)     #validate gets einsum code from ade list and handles
                                                             #permuations automatically
    #update user_data
    user_data['td'] = t * np.reciprocal(d) + user_data['td'] #generated amplitudes seed for next evolution

    #test convergence
    cycle_norm = np.linalg.norm(user_data['td'])
    if abs(cycle_norm - norm) < tol:
        converged = True
        break
    
    print('{:<15.10f}  {:<15.10f}'.format(cycle_norm, abs(cycle_norm - norm)))
    norm = cycle_norm
    
if converged:
    #get the energy
    E = expansion('D', 'E')
    V = validate(E, 'hf')                                     #new instance for different ade list
    energy_correction = V.evaluate('E_{1}', user_data)
    print('HF CCD energy correction is {:<10.6f} Hartree'.format(energy_correction))

0.5048370301     0.5048370301   
0.5350832715     0.0302462414   
0.5749348951     0.0398516236   
0.5918903905     0.0169554954   
0.6005635406     0.0086731501   
0.6047685304     0.0042049898   
0.6068430272     0.0020744968   
0.6078606123     0.0010175852   
0.6083614319     0.0005008196   
0.6086077304     0.0002462985   
0.6087289979     0.0001212675   
0.6087887081     0.0000597102   
0.6088181245     0.0000294164   
0.6088326194     0.0000144948   
0.6088397639     0.0000071445   
0.6088432861     0.0000035222   
0.6088450229     0.0000017368   
HF CCD energy correction is -0.169138  Hartree


Which agrees with reference sources. We are using eri and Fock values from the saved hydrogen fluoride data together with _cc-ade_ generated doubles amplitudes which are passed to an eval expression for evaluation. The eval expression evaluated is obtained from the _code_ and _permutation_ keys of the diagram list generated by _expansion_. This is not an efficient way of getting the amplitudes but has the advantage that the contribution of each diagram could be easily monitored. Additionally for higher orders where evaluation of individual diagrams can take time, it would be possible to detect diagrams which are only contributing values below some threshold and remove them from the active list. This 'selected' approach may be worthwhile as the logic tests needed to implement it are negligible compared to evaluation of quadruple expansion terms involving loops to a depth of eight. 

Also provided is a way of using the __expansion__ class instantisation to deal with a single contraction by invoking the class as __expansion(contraction=s_strings)__. To evaluate, for example, the triples addition to the singles (diagram $S_7$) we would use _expansion(contraction='++--:+++---')_

In [10]:
s = expansion(contraction='++--:+++---')
print(s.get('r', 'S_{7}'))
V = validate(s, 'c')
print()
V.compare('S_{7}')

{'id': '0010(hhpp)', 'labels': ('klcd', 'adcikl'), 'sign': (-1, [3, 2]), 'factor': 2, 'sigma': 'dklc', 'permutation': '', 'latex': '-\\frac{1}{4} \\displaystyle \\sum_{dklc} \\langle kl\\Vertcd \\rangle t^{adc}_{ikl}', 'code': "-0.25 * np.einsum('klcd,adcikl->ai', gs[o,o,v,v], tt)", 'reference': 'S_{7}', 'contraction': '++--'}

S_{7}     ++--         0.001552802417424   0.001552802417424  True True


Using the class invocation __expansion(order=n, series='latex series')__ it is possible to explore custom series or individual terms). The *latex_series* should have each term seperated by ' + ' and powers entered explicitly Eg '1 + T_1 + T_1 T_1 + \\\\frac{1}{2} T_2 T_2 T_1'

As an example Linear CCD

In [11]:
ld = expansion(order='D', series='1 + T_2')
ld.order_by_reference()
for i in ld.ade:
    print('{:6}    {:20}  {:10}'.format(i['reference'], i['code'], i['permutation']))

D_{1}     np.einsum('abij->abij', gs[v,v,o,o])            
D_{2a}    np.einsum('bc,acij->abij', fs[v,v], td)  \hat{P}(ab) 
D_{2b}    -np.einsum('kj,abik->abij', fs[o,o], td)  \hat{P}(ij) 
D_{2c}    0.5 * np.einsum('abcd,cdij->abij', gs[v,v,v,v], td)            
D_{2d}    0.5 * np.einsum('klij,abkl->abij', gs[o,o,o,o], td)            
D_{2e}    np.einsum('akic,cbkj->abij', gs[v,o,o,v], td)  \hat{P}(ab) \hat{P}(ij) 


Try

```expansion(order='D', series='1 + T_2')``` for LCCD

```expansion(order='S', series='1 + T_1 + T_2')``` and ```expansion(order='D', series='1 + T_1 + T_2')```
for LCCSD